In [6]:
!pip install kagglehub
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 48.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 74.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]


In [20]:
# Imports Dataset Kaggle
import kagglehub
# Download latest version
path = kagglehub.dataset_download("mariumfaheem666/spam-sms-classification-using-nlp")
file = "Spam_SMS.csv"
import pandas as pd

In [21]:
pd.read_csv(path+"/"+file)

,Class,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will ü b going to esplanade fr home?
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


In [ ]:
# Definição de Classes
class Mensagem:
    def __init__(self, mensagem_original, classificacao):
        self.MensagemOriginal = mensagem_original
        self.Classificacao = classificacao
        self.MensagemTratada = None
        self.MensagemTokenizada = None
        self.QuantidadeCaracteres = 0
        self.QuantidadePalavras = 0
        self.QuantidadeVerbos = 0
        self.QuantidadeSubstantivos = 0
        self.ArvoreSintagmas = None

    def ConstroiArvoreSintagmas(self):
        # Lógica para construir a árvore de sintagmas
        pass


class BaseMensagens:
    def __init__(self):
        self.BaseMensagens = []  # Lista de objetos Mensagem

    def Exec(self):
        # Lógica para execução genérica
        pass


class BaseTreinamento(BaseMensagens):
    def __init__(self):
        super().__init__()

    def Exec(self):
        # Lógica específica de treinamento
        pass

class BaseTeste(BaseMensagens):
    def __init__(self):
        super().__init__()
        self.Acuracidade = 0.0

    def Exec(self):
        # Lógica específica de teste
        pass
